<a href="https://colab.research.google.com/github/jhonycq/Practicas-python-pyspark/blob/main/Practica_pyspark_jhony.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 50.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=38f05cc4004302df17846b0a4204a8aa1a6108793ecf7b636770ffd03b32ff04
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

import datetime

spark = SparkSession.builder.appName("jhony").config("spark.jars", "postgresql-42.3.3.jar").getOrCreate()

servidor="34.132.209.74" 
puerto = 5432 
db = "base-datos-spark" 
usuario= "matthew" 
contraseña = "matthew" 
tabla_customers="customers" 


url = f"jdbc:postgresql://{servidor}/{db}"

customers_df = spark.read \
	.format("jdbc") \
    .option("url", url)\
    .option("dbtable", tabla_customers)\
    .option("user", usuario)\
    .option("password", contraseña)\
    .option("driver", "org.postgresql.Driver")\
    .load()

ruta_bucket = f"/content/archivos/test/customers/"
customers_df.repartition(4).write.mode('overwrite').option("compression","gzip").parquet(ruta_bucket)

In [ ]:
# tabla de suppliers (proveedores)
suppliers_df = spark.read \
	.format("jdbc") \
    .option("url", url)\
    .option("dbtable", "suppliers")\
    .option("user", usuario)\
    .option("password", contraseña)\
    .option("driver", "org.postgresql.Driver")\
    .load()

# tabla employees (empleados)
Employees_df = spark.read \
	.format("jdbc") \
    .option("url", url)\
    .option("dbtable", "employees")\
    .option("user", usuario)\
    .option("password", contraseña)\
    .option("driver", "org.postgresql.Driver")\
    .load()

# tabla products (productos)
products_df = spark.read \
	.format("jdbc") \
    .option("url", url)\
    .option("dbtable", "products")\
    .option("user", usuario)\
    .option("password", contraseña)\
    .option("driver", "org.postgresql.Driver")\
    .load()
# tabla orders (ordenes)
orders_df = spark.read \
	.format("jdbc") \
    .option("url", url)\
    .option("dbtable", "orders")\
    .option("user", usuario)\
    .option("password", contraseña)\
    .option("driver", "org.postgresql.Driver")\
    .load()

In [ ]:
# 1. Seleccionar los clientes que viven en el país de "usa"
customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- contact_title: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)



In [ ]:
customers_df.filter(customers_df.country == 'USA').show(5)

+-----------+--------------------+-------------+--------------------+--------------------+-------------+------+-----------+-------+--------------+--------------+
|customer_id|        company_name| contact_name|       contact_title|             address|         city|region|postal_code|country|         phone|           fax|
+-----------+--------------------+-------------+--------------------+--------------------+-------------+------+-----------+-------+--------------+--------------+
|      GREAL|Great Lakes Food ...|Howard Snyder|   Marketing Manager|    2732 Baker Blvd.|       Eugene|    OR|      97403|    USA|(503) 555-7555|          null|
|      HUNGC|Hungry Coyote Imp...|Yoshi Latimer|Sales Representative|City Center Plaza...|        Elgin|    OR|      97827|    USA|(503) 555-6874|(503) 555-2376|
|      LAZYK|Lazy K Kountry Store|   John Steel|   Marketing Manager|12 Orchestra Terrace|  Walla Walla|    WA|      99362|    USA|(509) 555-7969|(509) 555-6221|
|      LETSS|   Let's Stop N

In [ ]:
# 2. Seleccionar los proveedores que viven en la ciudad de "BERLIN"
suppliers_df.printSchema()

root
 |-- supplier_id: short (nullable = true)
 |-- company_name: string (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- contact_title: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)
 |-- homepage: string (nullable = true)



In [ ]:
suppliers_df.filter(suppliers_df.city == 'Berlin').show(truncate=False)

+-----------+---------------------------+-------------+-------------+------------------+------+------+-----------+-------+-------------+----+--------+
|supplier_id|company_name               |contact_name |contact_title|address           |city  |region|postal_code|country|phone        |fax |homepage|
+-----------+---------------------------+-------------+-------------+------------------+------+------+-----------+-------+-------------+----+--------+
|11         |Heli Süßwaren GmbH & Co. KG|Petra Winkler|Sales Manager|Tiergartenstraße 5|Berlin|null  |10785      |Germany|(010) 9984510|null|null    |
+-----------+---------------------------+-------------+-------------+------------------+------+------+-----------+-------+-------------+----+--------+



In [ ]:
# 3. Seleccionar los empleados con código 3,5 y 8
Employees_df.printSchema()

root
 |-- employee_id: short (nullable = true)
 |-- last_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- title_of_courtesy: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- home_phone: string (nullable = true)
 |-- extension: string (nullable = true)
 |-- photo: binary (nullable = true)
 |-- notes: string (nullable = true)
 |-- reports_to: short (nullable = true)
 |-- photo_path: string (nullable = true)



In [ ]:
Employees_df.filter(Employees_df.employee_id.isin([3,5,8])).show()

+-----------+---------+----------+--------------------+-----------------+----------+----------+--------------------+--------+------+-----------+-------+--------------+---------+-----+--------------------+----------+--------------------+
|employee_id|last_name|first_name|               title|title_of_courtesy|birth_date| hire_date|             address|    city|region|postal_code|country|    home_phone|extension|photo|               notes|reports_to|          photo_path|
+-----------+---------+----------+--------------------+-----------------+----------+----------+--------------------+--------+------+-----------+-------+--------------+---------+-----+--------------------+----------+--------------------+
|          3|Leverling|     Janet|Sales Representative|              Ms.|1963-08-30|1992-04-01|  722 Moss Bay Blvd.|Kirkland|    WA|      98033|    USA|(206) 555-3412|     3355|   []|Janet has a BS de...|         2|http://accweb/emm...|
|          5| Buchanan|    Steven|       Sales Manag

In [ ]:
# 5. Seleccionar los productos con precio mayor o igual a 20 y menor o igual a 90
products_df.printSchema()

root
 |-- product_id: short (nullable = true)
 |-- product_name: string (nullable = true)
 |-- supplier_id: short (nullable = true)
 |-- category_id: short (nullable = true)
 |-- quantity_per_unit: string (nullable = true)
 |-- unit_price: float (nullable = true)
 |-- units_in_stock: short (nullable = true)
 |-- units_on_order: short (nullable = true)
 |-- reorder_level: short (nullable = true)
 |-- discontinued: integer (nullable = true)



In [ ]:
products_df.filter(products_df.unit_price.between(20,90)).show()

+----------+--------------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|         4|Chef Anton's Caju...|          2|          2|    48 - 6 oz jars|      22.0|            53|             0|            0|           0|
|         5|Chef Anton's Gumb...|          2|          2|          36 boxes|     21.35|             0|             0|            0|           1|
|         6|Grandma's Boysenb...|          3|          2|    12 - 8 oz jars|      25.0|           120|             0|           25|           0|
|         7|Uncle Bob's Organ...|          3|          7|   12 - 1 lb pkgs.|      30.0|            15|             0|           10

In [ ]:
# 6. Mostrar las órdenes de compra entre las fechas 01/01/1997 al 15/07/1997
orders_df.printSchema()

root
 |-- order_id: short (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- employee_id: short (nullable = true)
 |-- order_date: date (nullable = true)
 |-- required_date: date (nullable = true)
 |-- shipped_date: date (nullable = true)
 |-- ship_via: short (nullable = true)
 |-- freight: float (nullable = true)
 |-- ship_name: string (nullable = true)
 |-- ship_address: string (nullable = true)
 |-- ship_city: string (nullable = true)
 |-- ship_region: string (nullable = true)
 |-- ship_postal_code: string (nullable = true)
 |-- ship_country: string (nullable = true)



In [ ]:
orders_df.show(3)

+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+------------------+--------------+-----------+----------------+------------+
|order_id|customer_id|employee_id|order_date|required_date|shipped_date|ship_via|freight|           ship_name|      ship_address|     ship_city|ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+------------------+--------------+-----------+----------------+------------+
|   10248|      VINET|          5|1996-07-04|   1996-08-01|  1996-07-16|       3|  32.38|Vins et alcools C...|59 rue de l'Abbaye|         Reims|       null|           51100|      France|
|   10249|      TOMSP|          6|1996-07-05|   1996-08-16|  1996-07-10|       1|  11.61|  Toms Spezialitäten|     Luisenstr. 48|       Münster|       null|           44087|     Germany|
|   10250|      HANAR|          4|1996-07-08|   1996-08-05|  1996

In [ ]:
orders_df.filter(orders_df.order_date.between('1997-01-01', ' 1997-07-15')).show()

+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+---------------+-------------+----------------+------------+
|order_id|customer_id|employee_id|order_date|required_date|shipped_date|ship_via|freight|           ship_name|        ship_address|      ship_city|  ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+---------------+-------------+----------------+------------+
|   10400|      EASTC|          1|1997-01-01|   1997-01-29|  1997-01-16|       3|  83.93|  Eastern Connection|      35 King George|         London|         null|         WX3 6FW|          UK|
|   10401|      RATTC|          1|1997-01-01|   1997-01-29|  1997-01-10|       1|  12.51|Rattlesnake Canyo...|     2817 Milton Dr.|    Albuquerque|           NM|           87110|         USA|
|   10402|      ERNSH|          8|1997-0

In [ ]:
#7. Mostrar las órdenes de compra hechas en el año 1997, que pertenecen a los empleados con códigos 1 ,3 ,4 ,8
# importaremos la funciones de sql a pyspark

from pyspark.sql import functions as F 

orders_df.show(3)

+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+------------------+--------------+-----------+----------------+------------+
|order_id|customer_id|employee_id|order_date|required_date|shipped_date|ship_via|freight|           ship_name|      ship_address|     ship_city|ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+------------------+--------------+-----------+----------------+------------+
|   10248|      VINET|          5|1996-07-04|   1996-08-01|  1996-07-16|       3|  32.38|Vins et alcools C...|59 rue de l'Abbaye|         Reims|       null|           51100|      France|
|   10249|      TOMSP|          6|1996-07-05|   1996-08-16|  1996-07-10|       1|  11.61|  Toms Spezialitäten|     Luisenstr. 48|       Münster|       null|           44087|     Germany|
|   10250|      HANAR|          4|1996-07-08|   1996-08-05|  1996

In [ ]:
orders_df.filter((F.year(orders_df.order_date)=='1997') & (orders_df.employee_id.isin(1,3,4,8))).show(6)

+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+---------------+-------------+----------------+------------+
|order_id|customer_id|employee_id|order_date|required_date|shipped_date|ship_via|freight|           ship_name|        ship_address|      ship_city|  ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+---------------+-------------+----------------+------------+
|   10400|      EASTC|          1|1997-01-01|   1997-01-29|  1997-01-16|       3|  83.93|  Eastern Connection|      35 King George|         London|         null|         WX3 6FW|          UK|
|   10401|      RATTC|          1|1997-01-01|   1997-01-29|  1997-01-10|       1|  12.51|Rattlesnake Canyo...|     2817 Milton Dr.|    Albuquerque|           NM|           87110|         USA|
|   10402|      ERNSH|          8|1997-0

In [ ]:
# 8. Mostrar las ordenes hechas en el año 1996

orders_df.filter(F.year(orders_df.order_date)== '1996').show(4)

+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+------------------+--------------+-----------+----------------+------------+
|order_id|customer_id|employee_id|order_date|required_date|shipped_date|ship_via|freight|           ship_name|      ship_address|     ship_city|ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+------------------+--------------+-----------+----------------+------------+
|   10248|      VINET|          5|1996-07-04|   1996-08-01|  1996-07-16|       3|  32.38|Vins et alcools C...|59 rue de l'Abbaye|         Reims|       null|           51100|      France|
|   10249|      TOMSP|          6|1996-07-05|   1996-08-16|  1996-07-10|       1|  11.61|  Toms Spezialitäten|     Luisenstr. 48|       Münster|       null|           44087|     Germany|
|   10250|      HANAR|          4|1996-07-08|   1996-08-05|  1996

In [ ]:
# 9. Mostrar las ordenes hechas en el año 1997 ,del mes de abril

orders_df.filter(orders_df.order_date.between('1997-04-01', ' 1997-04-30')).show(5)

+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+---------+-----------+----------------+------------+
|order_id|customer_id|employee_id|order_date|required_date|shipped_date|ship_via|freight|           ship_name|        ship_address|ship_city|ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+---------+-----------+----------------+------------+
|   10492|      BOTTM|          3|1997-04-01|   1997-04-29|  1997-04-11|       1|  62.89|Bottom-Dollar Mar...|  23 Tsawassen Blvd.|Tsawassen|         BC|         T2F 8M4|      Canada|
|   10493|      LAMAI|          4|1997-04-02|   1997-04-30|  1997-04-10|       3|  10.64|    La maison d'Asie|1 rue Alsace-Lorr...| Toulouse|       null|           31000|      France|
|   10494|      COMMI|          4|1997-04-02|   1997-04-30|  1997-04-09|       2

In [ ]:
# 10. Mostrar las ordenes hechas el primero de todos los meses, del año 1998
orders_df.filter((F.year(orders_df.order_date)=='1998') & (F.dayofmonth(orders_df.order_date)=='1')).show()

+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+------------+-----------+----------------+------------+
|order_id|customer_id|employee_id|order_date|required_date|shipped_date|ship_via|freight|           ship_name|        ship_address|   ship_city|ship_region|ship_postal_code|ship_country|
+--------+-----------+-----------+----------+-------------+------------+--------+-------+--------------------+--------------------+------------+-----------+----------------+------------+
|   10808|      OLDWO|          2|1998-01-01|   1998-01-29|  1998-01-09|       3|  45.53|Old World Delicat...|     2743 Bering St.|   Anchorage|         AK|           99508|         USA|
|   10809|      WELLI|          7|1998-01-01|   1998-01-29|  1998-01-07|       1|   4.87|Wellington Import...|  Rua do Mercado, 12|     Resende|         SP|       08737-363|      Brazil|
|   10810|      LAUGB|          2|1998-01-01|   1998-01-29|  1998

In [ ]:
# 11. Mostrar todos los clientes que no tienen fax
customers_df.show(2)

+-----------+--------------------+------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
|customer_id|        company_name|contact_name|       contact_title|             address|       city|region|postal_code|country|       phone|         fax|
+-----------+--------------------+------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
|      ALFKI| Alfreds Futterkiste|Maria Anders|Sales Representative|       Obere Str. 57|     Berlin|  null|      12209|Germany| 030-0074321| 030-0076545|
|      ANATR|Ana Trujillo Empa...|Ana Trujillo|               Owner|Avda. de la Const...|México D.F.|  null|      05021| Mexico|(5) 555-4729|(5) 555-3745|
+-----------+--------------------+------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
only showing top 2 rows



In [ ]:
customers_df.filter(customers_df.fax.isNull()).show(5)

+-----------+--------------------+-----------------+--------------------+--------------------+-----------+------+-----------+-----------+--------------+----+
|customer_id|        company_name|     contact_name|       contact_title|             address|       city|region|postal_code|    country|         phone| fax|
+-----------+--------------------+-----------------+--------------------+--------------------+-----------+------+-----------+-----------+--------------+----+
|      ANTON|Antonio Moreno Ta...|   Antonio Moreno|               Owner|     Mataderos  2312|México D.F.|  null|      05023|     Mexico|  (5) 555-3932|null|
|      BSBEV|       B's Beverages|Victoria Ashworth|Sales Representative|   Fauntleroy Circus|     London|  null|    EC2 5NT|         UK|(171) 555-1212|null|
|      CHOPS|   Chop-suey Chinese|        Yang Wang|               Owner|        Hauptstr. 29|       Bern|  null|       3012|Switzerland|   0452-076545|null|
|      COMMI|    Comércio Mineiro|     Pedro Afonso|

In [ ]:
# 12. Mostrar todos los clientes que tienen fax

customers_df.filter(customers_df.fax.isNotNull()).show(5)

+-----------+--------------------+------------------+--------------------+--------------------+-----------+------+-----------+-------+--------------+--------------+
|customer_id|        company_name|      contact_name|       contact_title|             address|       city|region|postal_code|country|         phone|           fax|
+-----------+--------------------+------------------+--------------------+--------------------+-----------+------+-----------+-------+--------------+--------------+
|      ALFKI| Alfreds Futterkiste|      Maria Anders|Sales Representative|       Obere Str. 57|     Berlin|  null|      12209|Germany|   030-0074321|   030-0076545|
|      ANATR|Ana Trujillo Empa...|      Ana Trujillo|               Owner|Avda. de la Const...|México D.F.|  null|      05021| Mexico|  (5) 555-4729|  (5) 555-3745|
|      AROUT|     Around the Horn|      Thomas Hardy|Sales Representative|     120 Hanover Sq.|     London|  null|    WA1 1DP|     UK|(171) 555-7788|(171) 555-6750|
|      BER

In [ ]:
# 13. Mostrar el nombre del producto, el precio, el stock y el nombre de la categoría a la que pertenece.

# tabla categories (categoria)
categories_df = spark.read \
	.format("jdbc") \
    .option("url", url)\
    .option("dbtable", "categories")\
    .option("user", usuario)\
    .option("password", contraseña)\
    .option("driver", "org.postgresql.Driver")\
    .load()


categories_df.show(truncate=False)

+-----------+--------------+----------------------------------------------------------+-------+
|category_id|category_name |description                                               |picture|
+-----------+--------------+----------------------------------------------------------+-------+
|1          |Beverages     |Soft drinks, coffees, teas, beers, and ales               |[]     |
|2          |Condiments    |Sweet and savory sauces, relishes, spreads, and seasonings|[]     |
|3          |Confections   |Desserts, candies, and sweet breads                       |[]     |
|4          |Dairy Products|Cheeses                                                   |[]     |
|5          |Grains/Cereals|Breads, crackers, pasta, and cereal                       |[]     |
|6          |Meat/Poultry  |Prepared meats                                            |[]     |
|7          |Produce       |Dried fruit and bean curd                                 |[]     |
|8          |Seafood       |Seaweed and 

In [ ]:
products_df.show(3)

+----------+-------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
|product_id| product_name|supplier_id|category_id|  quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+-------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
|         1|         Chai|          8|          1| 10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|        Chang|          1|          1| 24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
|         3|Aniseed Syrup|          1|          2|12 - 550 ml bottles|      10.0|            13|            70|           25|           0|
+----------+-------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
only showing top 3 rows



In [ ]:
df= products_df.join(categories_df, on=['category_id'], how='inner').select(products_df.product_name, 
                                                                        products_df.unit_price,
                                                                         products_df.units_in_stock,
                                                                         categories_df.category_name)

df.show()

+--------------------+----------+--------------+-------------+
|        product_name|unit_price|units_in_stock|category_name|
+--------------------+----------+--------------+-------------+
|                Chai|      18.0|            39|    Beverages|
|               Chang|      19.0|            17|    Beverages|
|  Guaraná Fantástica|       4.5|            20|    Beverages|
|       Sasquatch Ale|      14.0|           111|    Beverages|
|      Steeleye Stout|      18.0|            20|    Beverages|
|       Côte de Blaye|     263.5|            17|    Beverages|
|    Chartreuse verte|      18.0|            69|    Beverages|
|         Ipoh Coffee|      46.0|            17|    Beverages|
|Laughing Lumberja...|      14.0|            52|    Beverages|
|       Outback Lager|      15.0|            15|    Beverages|
|Rhönbräu Klosterbier|      7.75|           125|    Beverages|
|        Lakkalikööri|      18.0|            57|    Beverages|
|       Aniseed Syrup|      10.0|            13|   Cond

In [ ]:
# 14. Mostrar el nombre del producto, el precio producto, el código del proveedor y el nombre de la compañía proveedora.

suppliers_df.show(3)

+-----------+--------------------+----------------+--------------------+--------------+-----------+------+-----------+-------+--------------+--------------+-----------+
|supplier_id|        company_name|    contact_name|       contact_title|       address|       city|region|postal_code|country|         phone|           fax|   homepage|
+-----------+--------------------+----------------+--------------------+--------------+-----------+------+-----------+-------+--------------+--------------+-----------+
|          1|      Exotic Liquids|Charlotte Cooper|  Purchasing Manager|49 Gilbert St.|     London|  null|    EC1 4SD|     UK|(171) 555-2222|          null|       null|
|          2|New Orleans Cajun...|   Shelley Burke| Order Administrator|P.O. Box 78934|New Orleans|    LA|      70117|    USA|(100) 555-4822|          null|#CAJUN.HTM#|
|          3|Grandma Kelly's H...|   Regina Murphy|Sales Representative|707 Oxford Rd.|  Ann Arbor|    MI|      48104|    USA|(313) 555-5735|(313) 555-3349

In [ ]:
products_df.join(suppliers_df, on=['supplier_id'], how='inner').select(products_df.product_name,
                                                                       products_df.unit_price,
                                                                       suppliers_df.supplier_id,
                                                                       suppliers_df.company_name).show()

+--------------------+----------+-----------+--------------------+
|        product_name|unit_price|supplier_id|        company_name|
+--------------------+----------+-----------+--------------------+
|               Chang|      19.0|          1|      Exotic Liquids|
|       Aniseed Syrup|      10.0|          1|      Exotic Liquids|
|Chef Anton's Caju...|      22.0|          2|New Orleans Cajun...|
|Chef Anton's Gumb...|     21.35|          2|New Orleans Cajun...|
|Louisiana Fiery H...|     21.05|          2|New Orleans Cajun...|
|Louisiana Hot Spi...|      17.0|          2|New Orleans Cajun...|
|Grandma's Boysenb...|      25.0|          3|Grandma Kelly's H...|
|Uncle Bob's Organ...|      30.0|          3|Grandma Kelly's H...|
|Northwoods Cranbe...|      40.0|          3|Grandma Kelly's H...|
|     Mishi Kobe Niku|      97.0|          4|       Tokyo Traders|
|               Ikura|      31.0|          4|       Tokyo Traders|
|       Longlife Tofu|      10.0|          4|       Tokyo Trad

In [ ]:
# 15. Mostrar el número de orden, el código del producto, el precio, la cantidad y el total pagado por producto.

# tabla order details (detalles del pedido)
order_details = spark.read \
	.format("jdbc") \
    .option("url", url)\
    .option("dbtable", "order_details")\
    .option("user", usuario)\
    .option("password", contraseña)\
    .option("driver", "org.postgresql.Driver")\
    .load()

In [ ]:
order_details.show(3)

+--------+----------+----------+--------+--------+
|order_id|product_id|unit_price|quantity|discount|
+--------+----------+----------+--------+--------+
|   10248|        11|      14.0|      12|     0.0|
|   10248|        42|       9.8|      10|     0.0|
|   10248|        72|      34.8|       5|     0.0|
+--------+----------+----------+--------+--------+
only showing top 3 rows



In [ ]:
order_details.select(order_details.order_id,
                     order_details.product_id, 
                     order_details.unit_price, 
                     order_details.quantity,
                     (order_details.unit_price*order_details.quantity).alias('TOTAL PAGADO')).show(3)

+--------+----------+----------+--------+------------+
|order_id|product_id|unit_price|quantity|TOTAL PAGADO|
+--------+----------+----------+--------+------------+
|   10248|        11|      14.0|      12|       168.0|
|   10248|        42|       9.8|      10|        98.0|
|   10248|        72|      34.8|       5|       174.0|
+--------+----------+----------+--------+------------+
only showing top 3 rows



In [ ]:
# 16. Mostrar el número de la orden, fecha, código del producto, precio, código del empleado y su nombre completo.

order_details.show(3)
Employees_df.show(3)
orders_df.show(3)

+--------+----------+----------+--------+--------+
|order_id|product_id|unit_price|quantity|discount|
+--------+----------+----------+--------+--------+
|   10248|        11|      14.0|      12|     0.0|
|   10248|        42|       9.8|      10|     0.0|
|   10248|        72|      34.8|       5|     0.0|
+--------+----------+----------+--------+--------+
only showing top 3 rows

+-----------+---------+----------+--------------------+-----------------+----------+----------+--------------------+--------+------+-----------+-------+--------------+---------+-----+--------------------+----------+--------------------+
|employee_id|last_name|first_name|               title|title_of_courtesy|birth_date| hire_date|             address|    city|region|postal_code|country|    home_phone|extension|photo|               notes|reports_to|          photo_path|
+-----------+---------+----------+--------------------+-----------------+----------+----------+--------------------+--------+------+-----------+

In [ ]:
# 17. Mostrar los 10 productos con menor stock

products_df.show(2)

+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|product_id|product_name|supplier_id|category_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|         1|        Chai|          8|          1|10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|       Chang|          1|          1|24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
only showing top 2 rows



In [ ]:
products_df.orderBy(products_df.units_in_stock.asc()).show(10)

+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|        29|Thüringer Rostbra...|         12|          6|50 bags x 30 sausgs.|    123.79|             0|             0|            0|           1|
|         5|Chef Anton's Gumb...|          2|          2|            36 boxes|     21.35|             0|             0|            0|           1|
|        31|   Gorgonzola Telino|         14|          4|     12 - 100 g pkgs|      12.5|             0|            70|           20|           0|
|        17|        Alice Mutton|          7|          6|      20 - 1 kg tins|      39.0|             0|             0

In [ ]:
# 18. Mostrar los 10 productos con mayor stock
products_df.orderBy(products_df.units_in_stock.desc()).show(10)


+----------+--------------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|  quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
|        75|Rhönbräu Klosterbier|         12|          1| 24 - 0.5 l bottles|      7.75|           125|             0|           25|           0|
|        40|    Boston Crab Meat|         19|          8|     24 - 4 oz tins|      18.4|           123|             0|           30|           0|
|         6|Grandma's Boysenb...|          3|          2|     12 - 8 oz jars|      25.0|           120|             0|           25|           0|
|        55|        Pâté chinois|         25|          6|  24 boxes x 2 pies|      24.0|           115|             0|      

In [ ]:
# 19. Mostrar los 10 productos con menor precio
products_df.orderBy(products_df.unit_price.asc()).show(10)


+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|        33|             Geitost|         15|          4|               500 g|       2.5|           112|             0|           20|           0|
|        24|  Guaraná Fantástica|         10|          1|    12 - 355 ml cans|       4.5|            20|             0|            0|           1|
|        13|               Konbu|          6|          8|            2 kg box|       6.0|            24|             0|            5|           0|
|        52|            Filo Mix|         24|          5|     16 - 2 kg boxes|       7.0|            38|             0

In [ ]:
# 20. Mostrar los 10 productos con mayor precio
products_df.orderBy(products_df.unit_price.desc()).show(10)

+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|        38|       Côte de Blaye|         18|          1|  12 - 75 cl bottles|     263.5|            17|             0|           15|           0|
|        29|Thüringer Rostbra...|         12|          6|50 bags x 30 sausgs.|    123.79|             0|             0|            0|           1|
|         9|     Mishi Kobe Niku|          4|          6|    18 - 500 g pkgs.|      97.0|            29|             0|            0|           1|
|        20|Sir Rodney's Marm...|          8|          3|       30 gift boxes|      81.0|            40|             0

# ***PRACTICA DE INTRODUCCION A TRANSACT SQL***

In [ ]:
# 1. Seleccionar todos los campos de la tabla clientes,ordenar por compania

import pyspark.sql.functions as f

customers_df.show(2)

+-----------+--------------------+------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
|customer_id|        company_name|contact_name|       contact_title|             address|       city|region|postal_code|country|       phone|         fax|
+-----------+--------------------+------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
|      ALFKI| Alfreds Futterkiste|Maria Anders|Sales Representative|       Obere Str. 57|     Berlin|  null|      12209|Germany| 030-0074321| 030-0076545|
|      ANATR|Ana Trujillo Empa...|Ana Trujillo|               Owner|Avda. de la Const...|México D.F.|  null|      05021| Mexico|(5) 555-4729|(5) 555-3745|
+-----------+--------------------+------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
only showing top 2 rows



In [ ]:
customers_df.orderBy(customers_df.company_name).show()

+-----------+--------------------+------------------+--------------------+--------------------+------------+------+-----------+-----------+--------------+--------------+
|customer_id|        company_name|      contact_name|       contact_title|             address|        city|region|postal_code|    country|         phone|           fax|
+-----------+--------------------+------------------+--------------------+--------------------+------------+------+-----------+-----------+--------------+--------------+
|      ALFKI| Alfreds Futterkiste|      Maria Anders|Sales Representative|       Obere Str. 57|      Berlin|  null|      12209|    Germany|   030-0074321|   030-0076545|
|      ANATR|Ana Trujillo Empa...|      Ana Trujillo|               Owner|Avda. de la Const...| México D.F.|  null|      05021|     Mexico|  (5) 555-4729|  (5) 555-3745|
|      ANTON|Antonio Moreno Ta...|    Antonio Moreno|               Owner|     Mataderos  2312| México D.F.|  null|      05023|     Mexico|  (5) 555-3

In [ ]:
# 2. Seleccionar todos los campos de clientes,cuya compania empiece con la letra B y pertenezcan a UK ,ordenar por nombre de la compania

customers_df.show(3)

+-----------+--------------------+--------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
|customer_id|        company_name|  contact_name|       contact_title|             address|       city|region|postal_code|country|       phone|         fax|
+-----------+--------------------+--------------+--------------------+--------------------+-----------+------+-----------+-------+------------+------------+
|      ALFKI| Alfreds Futterkiste|  Maria Anders|Sales Representative|       Obere Str. 57|     Berlin|  null|      12209|Germany| 030-0074321| 030-0076545|
|      ANATR|Ana Trujillo Empa...|  Ana Trujillo|               Owner|Avda. de la Const...|México D.F.|  null|      05021| Mexico|(5) 555-4729|(5) 555-3745|
|      ANTON|Antonio Moreno Ta...|Antonio Moreno|               Owner|     Mataderos  2312|México D.F.|  null|      05023| Mexico|(5) 555-3932|        null|
+-----------+--------------------+--------------+---------

In [ ]:
customers_df.filter(f.col('company_name').like('B%') & (f.col('country')=='UK')).show()

+-----------+-------------+-----------------+--------------------+-----------------+------+------+-----------+-------+--------------+----+
|customer_id| company_name|     contact_name|       contact_title|          address|  city|region|postal_code|country|         phone| fax|
+-----------+-------------+-----------------+--------------------+-----------------+------+------+-----------+-------+--------------+----+
|      BSBEV|B's Beverages|Victoria Ashworth|Sales Representative|Fauntleroy Circus|London|  null|    EC2 5NT|     UK|(171) 555-1212|null|
+-----------+-------------+-----------------+--------------------+-----------------+------+------+-----------+-------+--------------+----+



In [ ]:
# 3. Seleccionar todos los campos de productos de las categorias 1,3 y 5,ordenar por categoria

products_df.show(2)

+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|product_id|product_name|supplier_id|category_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|         1|        Chai|          8|          1|10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|       Chang|          1|          1|24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
only showing top 2 rows



In [ ]:
df1=products_df.filter(products_df.category_id.isin(1,3,5))

df1.show()

+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|         1|                Chai|          8|          1|  10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|               Chang|          1|          1|  24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
|        16|             Pavlova|          7|          3|    32 - 500 g boxes|     17.45|            29|             0|           10|           0|
|        19|Teatime Chocolate...|          8|          3|10 boxes x 12 pieces|       9.2|            25|             0

In [ ]:
df1.orderBy(df1.category_id.asc()).show()

+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|        24|  Guaraná Fantástica|         10|          1|    12 - 355 ml cans|       4.5|            20|             0|            0|           1|
|         2|               Chang|          1|          1|  24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
|        75|Rhönbräu Klosterbier|         12|          1|  24 - 0.5 l bottles|      7.75|           125|             0|           25|           0|
|        76|        Lakkalikööri|         23|          1|              500 ml|      18.0|            57|             0

In [ ]:
# 4. Seleccionar los productos cuyos precios unitarios estan entre 50 y 200

products_df.filter(products_df.unit_price.between(50.0,200.0)).show()

+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|         9|     Mishi Kobe Niku|          4|          6|    18 - 500 g pkgs.|      97.0|            29|             0|            0|           1|
|        18|    Carnarvon Tigers|          7|          8|          16 kg pkg.|      62.5|            42|             0|            0|           0|
|        20|Sir Rodney's Marm...|          8|          3|       30 gift boxes|      81.0|            40|             0|            0|           0|
|        29|Thüringer Rostbra...|         12|          6|50 bags x 30 sausgs.|    123.79|             0|             0

In [ ]:
# 5. Visualizar el nombre y el id de la compania del cliente,fecha,precio unitario y producto de la orden
products_df.show(1)
order_details.show(1)
orders_df.show(1)
customers_df.show(1)

+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|product_id|product_name|supplier_id|category_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|         1|        Chai|          8|          1|10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
only showing top 1 row

+--------+----------+----------+--------+--------+
|order_id|product_id|unit_price|quantity|discount|
+--------+----------+----------+--------+--------+
|   10248|        11|      14.0|      12|     0.0|
+--------+----------+----------+--------+--------+
only showing top 1 row

+--------+--

In [ ]:
products_df.join(order_details, on=['product_id'], how='inner').join(orders_df, on=['order_id']).join(customers_df, on=['customer_id']).select(
    orders_df.order_id,
    customers_df.company_name,
    orders_df.order_date,
    order_details.unit_price,
    products_df.product_name).show(3)

+--------+--------------------+----------+----------+-----------------+
|order_id|        company_name|order_date|unit_price|     product_name|
+--------+--------------------+----------+----------+-----------------+
|   10253|       Hanari Carnes|1996-07-10|      10.0|Gorgonzola Telino|
|   10272|Rattlesnake Canyo...|1996-08-02|      10.0|Gorgonzola Telino|
|   10273|          QUICK-Stop|1996-08-05|      10.0|Gorgonzola Telino|
+--------+--------------------+----------+----------+-----------------+
only showing top 3 rows



In [ ]:
# 6. Visualizar el nombre de la categoria y el numero de productos que hay por cada categoria.

products_df.show(2)
categories_df.show(2)

+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|product_id|product_name|supplier_id|category_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|         1|        Chai|          8|          1|10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|       Chang|          1|          1|24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
only showing top 2 rows

+-----------+-------------+--------------------+-------+
|category_id|category_name|         description|picture|
+-----------+-------------+------------

In [ ]:
df_cat = categories_df.join(products_df, on=['category_id'], how='inner').show(2)



+-----------+-------------+--------------------+-------+----------+------------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|category_id|category_name|         description|picture|product_id|product_name|supplier_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+-----------+-------------+--------------------+-------+----------+------------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|          1|    Beverages|Soft drinks, coff...|     []|         1|        Chai|          8|10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|          1|    Beverages|Soft drinks, coff...|     []|         2|       Chang|          1|24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
+-----------+-------------+--------------------+-------+----------+------------+-----------+--------

In [ ]:
df_cat.count

In [ ]:
# 7. Seleccionar los 5 productos mas vendidos

In [ ]:
#8. Seleccionar los jefes de los empleados

In [ ]:
#9. Obtener todos los productos cuyo nombre comienzan con M y tienen un precio comprendido entre 28 y 129

products_df.filter(f.col('product_name').like('M%') & (products_df.unit_price.between(28,129))).show()

+----------+--------------------+-----------+-----------+-----------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+-----------------+----------+--------------+--------------+-------------+------------+
|         9|     Mishi Kobe Niku|          4|          6| 18 - 500 g pkgs.|      97.0|            29|             0|            0|           1|
|        32|  Mascarpone Fabioli|         14|          4| 24 - 200 g pkgs.|      32.0|             9|            40|           25|           0|
|        51|Manjimup Dried Ap...|         24|          7| 50 - 300 g pkgs.|      53.0|            20|             0|           10|           0|
|        72|Mozzarella di Gio...|         14|          4| 24 - 200 g pkgs.|      34.8|            14|             0|            0|      

In [ ]:
# 10. Obtener todos los clientes del Pais de USA,Francia y UK

customers_df.filter(customers_df.country.isin(['USA','UK','France'])).show(10)

+-----------+--------------------+------------------+--------------------+--------------------+----------+------+-----------+-------+--------------+--------------+
|customer_id|        company_name|      contact_name|       contact_title|             address|      city|region|postal_code|country|         phone|           fax|
+-----------+--------------------+------------------+--------------------+--------------------+----------+------+-----------+-------+--------------+--------------+
|      AROUT|     Around the Horn|      Thomas Hardy|Sales Representative|     120 Hanover Sq.|    London|  null|    WA1 1DP|     UK|(171) 555-7788|(171) 555-6750|
|      BLONP|Blondesddsl père ...|Frédérique Citeaux|   Marketing Manager|    24, place Kléber|Strasbourg|  null|      67000| France|   88.60.15.31|   88.60.15.32|
|      BONAP|            Bon app'|  Laurence Lebihan|               Owner|12, rue des Bouchers| Marseille|  null|      13008| France|   91.24.45.40|   91.24.45.41|
|      BSBEV|   

In [ ]:
# 11. Obtener todos los productos descontinuados o con stock cero.
products_df.show(2)



+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|product_id|product_name|supplier_id|category_id| quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
|         1|        Chai|          8|          1|10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|       Chang|          1|          1|24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
+----------+------------+-----------+-----------+------------------+----------+--------------+--------------+-------------+------------+
only showing top 2 rows



In [ ]:
products_df.filter((products_df.discontinued == 1) &  (products_df.units_in_stock ==0)).show(3)

+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|         5|Chef Anton's Gumb...|          2|          2|            36 boxes|     21.35|             0|             0|            0|           1|
|        17|        Alice Mutton|          7|          6|      20 - 1 kg tins|      39.0|             0|             0|            0|           1|
|        29|Thüringer Rostbra...|         12|          6|50 bags x 30 sausgs.|    123.79|             0|             0|            0|           1|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------

In [ ]:
# 12. Obtener todas las ordenes hechas por el empleado King Robert


In [ ]:
# 13. Obtener todas las ordenes por el cliente cuya compania es "Que delicia"

In [ ]:
# 14. Obtener todas las ordenes hechas por el empleado King Robert,Davolio Nancy y Fuller Andrew

In [ ]:
# 15. Obtener todos los productos(codigo,nombre,precio,stock) de la orden 10257

In [ ]:
#16. Obtener todos los productos(codigo,nombre,precio,stock) de las ordenes hechas desde 1997 hasta la fecha de hoy.

In [ ]:
# 17. Calcular los 15 productos mas caros

products_df.show(3)

+----------+-------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
|product_id| product_name|supplier_id|category_id|  quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+-------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
|         1|         Chai|          8|          1| 10 boxes x 30 bags|      18.0|            39|             0|           10|           1|
|         2|        Chang|          1|          1| 24 - 12 oz bottles|      19.0|            17|            40|           25|           1|
|         3|Aniseed Syrup|          1|          2|12 - 550 ml bottles|      10.0|            13|            70|           25|           0|
+----------+-------------+-----------+-----------+-------------------+----------+--------------+--------------+-------------+------------+
only showing top 3 rows



In [ ]:
products_df.orderBy(products_df.unit_price.desc()).show(15)

+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|product_id|        product_name|supplier_id|category_id|   quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+--------------------+-----------+-----------+--------------------+----------+--------------+--------------+-------------+------------+
|        38|       Côte de Blaye|         18|          1|  12 - 75 cl bottles|     263.5|            17|             0|           15|           0|
|        29|Thüringer Rostbra...|         12|          6|50 bags x 30 sausgs.|    123.79|             0|             0|            0|           1|
|         9|     Mishi Kobe Niku|          4|          6|    18 - 500 g pkgs.|      97.0|            29|             0|            0|           1|
|        20|Sir Rodney's Marm...|          8|          3|       30 gift boxes|      81.0|            40|             0

In [ ]:
# 18. Calcular los 5 productos mas baratos

products_df.orderBy(products_df.unit_price.asc()).show(5)

+----------+------------------+-----------+-----------+-----------------+----------+--------------+--------------+-------------+------------+
|product_id|      product_name|supplier_id|category_id|quantity_per_unit|unit_price|units_in_stock|units_on_order|reorder_level|discontinued|
+----------+------------------+-----------+-----------+-----------------+----------+--------------+--------------+-------------+------------+
|        33|           Geitost|         15|          4|            500 g|       2.5|           112|             0|           20|           0|
|        24|Guaraná Fantástica|         10|          1| 12 - 355 ml cans|       4.5|            20|             0|            0|           1|
|        13|             Konbu|          6|          8|         2 kg box|       6.0|            24|             0|            5|           0|
|        52|          Filo Mix|         24|          5|  16 - 2 kg boxes|       7.0|            38|             0|           25|           0|
|     

In [ ]:
# 19. Obtener el nombre de todas las categorias y los nombres de sus productos,precio y stock.


In [ ]:
# 20. Obtener el nombre de todas las categorias y los nombres de sus productos,solo los productos que su nombre no comience con la letra P

In [ ]:
# 21. Calcular el stock de productos por cada categoria.Mostrar el nombre de la categoria y el stock por categoria.

In [ ]:
# 22. Obtener el Nombre del cliente,Nombre del Proveedor,Nombre del empleado y el nombre de los productos que estan en la orden 10794

In [ ]:
# 23. Mostrar el numero de ordenes de cada uno de los clientes por año,luego ordenar codigo del cliente y el año.

In [ ]:
# 24. Contar el numero de ordenes que se han realizado por años y meses,luego debe ser ordenado por año y por mes.

In [ ]:
# 25. Seleccionar el nombre de la compañía del cliente,él código de la orden de compra,la fecha de la orden de compra, código del producto, 
# cantidad pedida del producto,nombre del producto, el nombre de la compañía proveedora y la ciudad del proveedor ,usar Join

In [ ]:
# 26. Seleccionar el nombre de la compañía del cliente, nombre del contacto, el código de la orden de compra, la fecha de la orden de compra, 
# el código del producto,cantidad pedida del producto, nombre del producto y el nombre de la compañía proveedora, usas JOIN.Solamente las 
# compañías proveedoras que comienzan con la letra de la A hasta la letra G,además la cantidad pedida del producto debe estar entre 23 y 187.